In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
bus2402_df = pd.read_csv("./Data/2024년_버스노선별_정류장별_시간대별_승하차_인원_정보(02월).csv")

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\141047399.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  bus2402_df = pd.read_csv("./Data/2024년_버스노선별_정류장별_시간대별_승하차_인원_정보(02월).csv")


In [56]:
busstop_df = pd.read_excel("./Data/서울시버스정류소위치정보(20240305).xlsx")

In [122]:
busstop_df.groupby(["ARS_ID", "정류소명"]).head()

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입
0,100000001,1001,종로2가사거리,126.987752,37.569806,중앙차로
1,100000002,1002,창경궁.서울대학교병원,126.996521,37.579433,중앙차로
2,100000003,1003,명륜3가.성대입구,126.998251,37.582580,중앙차로
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로
...,...,...,...,...,...,...
11256,124000334,25995,우성아파트,127.139339,37.550386,일반차로
11257,124000333,25996,우성아파트,127.140046,37.550643,일반차로
11258,124000332,25997,조일약국,127.123596,37.533630,일반차로
11259,124000331,25998,성내시장,127.125497,37.536155,일반차로


In [246]:
def ars_coord(ars):
    try:
        a = [busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 4].iloc[0], busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 3].iloc[0]]
        return a
        
    except:
        return ""

In [249]:
def station_coord(station):
    try:
        a = [busstop_df.loc[busstop_df["정류소명"] == station].iloc[:, 4].iloc[0], busstop_df.loc[busstop_df["정류소명"] == station].iloc[:, 3].iloc[0]]
        return a
        
    except:
        return ""

In [237]:
ars_coord(1002)

[37.579432932, 126.9965206101]

In [243]:
busstop_df[busstop_df["ARS_ID"] == 4835]

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입


In [221]:
def convert_to_int(value):
    if isinstance(value, int):
        return value
    else:
        return 0

In [247]:
bus2402_df2["ars_coord"] = bus2402_df2["버스정류장ARS번호"].map(convert_to_int).map(ars_coord)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\1546771542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["ars_coord"] = bus2402_df2["버스정류장ARS번호"].map(convert_to_int).map(ars_coord)


In [315]:
bus2402_df2["station_coord"] = bus2402_df2["역명3"].map(station_coord)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\2107388508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["station_coord"] = bus2402_df2["역명3"].map(station_coord)


In [300]:
bus2402_df2.loc[(bus2402_df2["ars_coord"] == "") & (bus2402_df2["station_coord"] == "")]

,버스정류장ARS번호,역명2,ars_coord,station_coord,역명3
15800,35331,은평공영차고지,,,은평공영차고지
19553,14148,난지한강공원(야구장),,,난지한강공원(야구장)
32743,0,염곡지하차도북측(가상),,,염곡지하차도북측
32918,22574,방배역.서초프라자약국,,,방배역.서초프라자약국
32948,22878,파이낸셜뉴스.현대렉시온오피스텔,,,파이낸셜뉴스.현대렉시온오피스텔
...,...,...,...,...,...
42341,~,은평공영차고지(가상),,,은평공영차고지
42352,~,김포교통(기점가상),,,김포교통
42358,~,김포교통(종점가상),,,김포교통
42364,~,구미동차고지(기점가상),,,구미동차고지


In [161]:
busstop_df.loc[busstop_df["ARS_ID"] == 1001].iloc[:, 4][0]

37.5698055407

In [120]:
pd.Series(bus2402_df["역명"].str.slice(stop=-7).unique()).isin(busstop_df["정류소명"])

0        True
1        True
2        True
3        True
4        True
        ...  
8149    False
8150    False
8151    False
8152    False
8153    False
Length: 8154, dtype: bool

In [140]:
bus2402_df2 = bus2402_df.loc[:,("버스정류장ARS번호", "역명2")]

In [143]:
bus2402_df2 = bus2402_df2.drop_duplicates()

In [314]:
busstop_df["정류소명"] = busstop_df["정류소명"].map(remove_string)

In [52]:
bus2402_df["역명"].str.slice(stop=-7).unique()

array(['종로2가사거리', '창경궁.서울대학교병원', '명륜3가.성대입구', ..., '김포교통(종점가상)',
       '구미동차고지(기점가상)', '가좌동종점(종점가상)'], dtype=object)

In [ ]:
url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"

request=coordsToaddr

params = {"query" : word}

headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
          "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}

res = requests.get(url, params = params, headers = headers)

In [21]:
def reverse_geocoding(latitude, longitude):
    url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={longitude},{latitude}&output=json"
    headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
          "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data.get('status') == 'OK':
            address = data['results'][0]['region']['area1']['name'] + ' ' + data['results'][0]['region']['area2']['name'] + ' ' + data['results'][0]['region']['area3']['name']
            return address
        else:
            print("Reverse geocoding failed.")
    else:
        print("Error:", response.status_code)

# 사용 예시
latitude = 37.5665  # 위도
longitude = 126.9780  # 경도

address = reverse_geocoding(latitude, longitude)
print("주소:", address)


Reverse geocoding failed.
주소: None


In [22]:
def reverse_geocoding(latitude, longitude, api_key_id, api_key):
    url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={longitude},{latitude}&output=json"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": api_key_id,
        "X-NCP-APIGW-API-KEY": api_key
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data.get('status') == 'OK':
            address = data['results'][0]['region']['area1']['name'] + ' ' + data['results'][0]['region']['area2']['name'] + ' ' + data['results'][0]['region']['area3']['name']
            return address
        else:
            print("Reverse geocoding failed.")
    else:
        print("Error:", response.status_code)

# 사용 예시
latitude = 37.5665  # 위도
longitude = 126.9780  # 경도
api_key_id = "0tsfmn4cxb"
api_key = "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"

address = reverse_geocoding(latitude, longitude, api_key_id, api_key)
print("주소:", address)


Reverse geocoding failed.
주소: None


In [31]:
curl = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=126.9877522923,37.5698055407&sourcecrs=epsg:4326&output=json&orders=legalcode,admcode"


In [24]:
headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
          "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}

In [32]:
res = requests.get(curl, headers)

In [37]:
res.json()

{'status': {'code': 0, 'name': 'ok', 'message': 'done'},
 'results': [{'name': 'legalcode',
   'code': {'id': '1111013800', 'type': 'L', 'mappingId': '09110138'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978388,
       'y': 37.56661}},
     'alias': '서울'},
    'area2': {'name': '종로구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978835,
       'y': 37.573521}}},
    'area3': {'name': '종로2가',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.986903,
       'y': 37.57044}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}},
  {'name': 'admcode',
   'code': {'id': '1111061500', 'type': 'A', 'mappingId': '09110615'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {

In [50]:
res.json()["results"][0]["region"]["area2"]["name"]

'종로구'

In [ ]:
{'status': {'code': 0, 'name': 'ok', 'message': 'done'},
 'results': [{'name': 'legalcode',
   'code': {'id': '1111013800', 'type': 'L', 'mappingId': '09110138'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978388,
       'y': 37.56661}},
     'alias': '서울'},
    'area2': {'name': '종로구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978835,
       'y': 37.573521}}},
    'area3': {'name': '종로2가',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.986903,
       'y': 37.57044}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}},
  {'name': 'admcode',
   'code': {'id': '1111061500', 'type': 'A', 'mappingId': '09110615'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978388,
       'y': 37.56661}},
     'alias': '서울'},
    'area2': {'name': '종로구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978835,
       'y': 37.573521}}},
    'area3': {'name': '종로1.2.3.4가동',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.990262,
       'y': 37.574461}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}}]}

In [264]:
bus2402_df.head()

,사용년월,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,...,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수,교통수단타입코드,교통수단타입명,등록일자,역명2
0,202402,741,741번(진관차고지~헌인릉입구),100000001,1001,종로2가사거리(00075),113,135,87,117,...,240,156,209,135,197,111,10,서울간선버스,20240303,종로2가사거리
1,202402,N37,N37번(송파공영차고지~진관공영차고지),100000001,1001,종로2가사거리(00032),9,17,160,177,...,0,0,0,0,0,0,51,서울심야버스,20240303,종로2가사거리
2,202402,470,470번(상암차고지~안골마을),100000001,1001,종로2가사거리(00066),79,88,0,1,...,353,235,305,192,221,106,10,서울간선버스,20240303,종로2가사거리
3,202402,N37,N37번(진관공영차고지~송파공영차고지),100000001,1001,종로2가사거리(00089),0,0,0,0,...,0,0,0,0,0,0,51,서울심야버스,20240303,종로2가사거리
4,202402,100,100번(하계동~용산구청),100000002,1002,창경궁.서울대학교병원(00031),0,0,0,0,...,32,22,54,35,24,3,10,서울간선버스,20240303,창경궁.서울대학교병원


In [265]:
busstop_coord_df = bus2402_df.loc[:, ("버스정류장ARS번호", "역명2")]

In [267]:
busstop_coord_df = busstop_coord_df.drop_duplicates()

In [269]:
busstop_coord_df["버스정류장ARS번호"] = busstop_coord_df["버스정류장ARS번호"].map(convert_to_int)

In [281]:
busstop_coord_df["버스정류장ARS번호"].isin(busstop_df["ARS_ID"])

0         True
4         True
23        True
42        True
49        True
         ...  
42341    False
42352    False
42358    False
42364    False
42367    False
Name: 버스정류장ARS번호, Length: 12701, dtype: bool

In [279]:
busstop_coord_df["역명2"].isin(busstop_df["정류소명"]).describe()

count     12701
unique        2
top        True
freq      10543
Name: 역명2, dtype: object

In [283]:
bus2402_df2.loc[(bus2402_df2["ars_coord"] == "") & (bus2402_df2["station_coord"] == "")]

,버스정류장ARS번호,역명2,ars_coord,station_coord
4445,4835,옥수역(가상),,
15800,35331,은평공영차고지,,
19553,14148,난지한강공원(야구장),,
32743,0,염곡지하차도북측(가상),,
32918,22574,방배역.서초프라자약국,,
...,...,...,...,...
42340,~,청암자이아파트(가상),,
42352,~,김포교통(기점가상),,
42358,~,김포교통(종점가상),,
42364,~,구미동차고지(기점가상),,


In [327]:
def station_coord(station):
    try:
        a = [busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 4].iloc[0], 
             busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 3].iloc[0]]
        return a
        
    except:
        return ""

In [334]:
busstop_df.loc[busstop_df["정류소명"].str.contains("국민은행")].iloc[:, 4].iloc[0]

37.6046629766

In [335]:
busstop_df.loc[busstop_df["정류소명"].str.contains("국민은행")].iloc[:, 3].iloc[0]

126.9608880061

In [332]:
busstop_df["정류소명"].str.contains("무역센터")

0        False
1        False
2        False
3        False
4        False
         ...  
11256    False
11257    False
11258    False
11259    False
11260    False
Name: 정류소명, Length: 11261, dtype: bool

In [343]:
bus2402_df2["station_coord"] = bus2402_df2["역명3"].map(station_coord)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\3080729665.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  a = [busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 4].iloc[0],
C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\3080729665.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 3].iloc[0]]
C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\2107388508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["station_coord"] = bus2402_df2["역명3"].map(station_coord)


In [289]:
remove_string = ["(가상)", "(기점가상)", "(종점가상)"]

In [294]:
def remove_string(text):
    for i in ["(가상)", "(기점가상)", "(종점가상)"]:
        text = text.replace(i, "")
        
    return text

In [297]:
bus2402_df2["역명3"] = bus2402_df2["역명2"].map(remove_string)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\651415754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["역명3"] = bus2402_df2["역명2"].map(remove_string)


In [298]:
bus2402_df2.head()

,버스정류장ARS번호,역명2,ars_coord,station_coord,역명3
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]","[37.5698055407, 126.9877522923]",종로2가사거리
4,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]","[37.579432932, 126.9965206101]",창경궁.서울대학교병원
23,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]","[37.5825800121, 126.9982508372]",명륜3가.성대입구
42,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]","[37.5685792736, 126.9876130976]",종로2가.삼일교
49,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]","[37.586243, 127.001744]",혜화동로터리.여운형활동터


In [344]:
bus2402_df2.loc[(bus2402_df2["ars_coord"] == "") & (bus2402_df2["station_coord"] == "")]

,버스정류장ARS번호,역명2,ars_coord,station_coord,역명3
19553,14148,난지한강공원(야구장),,,난지한강공원(야구장)
32743,0,염곡지하차도북측(가상),,,염곡지하차도북측
32918,22574,방배역.서초프라자약국,,,방배역서초프라자약국
32948,22878,파이낸셜뉴스.현대렉시온오피스텔,,,파이낸셜뉴스현대렉시온오피스텔
32966,22931,방배열린문화센터(방배4동주민센터),,,방배열린문화센터(방배4동주민센터)
...,...,...,...,...,...
42337,~,구미동차고지(종점가상),,,구미동차고지
42352,~,김포교통(기점가상),,,김포교통
42358,~,김포교통(종점가상),,,김포교통
42364,~,구미동차고지(기점가상),,,구미동차고지


In [338]:
bus2402_df2.loc[(bus2402_df2["ars_coord"] == "") & (bus2402_df2["station_coord"] == "")].iloc[0:20, :]

,버스정류장ARS번호,역명2,ars_coord,station_coord,역명3
19553,14148,난지한강공원(야구장),,,난지한강공원(야구장)
32743,0,염곡지하차도북측(가상),,,염곡지하차도북측
32918,22574,방배역.서초프라자약국,,,방배역.서초프라자약국
32948,22878,파이낸셜뉴스.현대렉시온오피스텔,,,파이낸셜뉴스.현대렉시온오피스텔
32966,22931,방배열린문화센터(방배4동주민센터),,,방배열린문화센터(방배4동주민센터)
33127,22981,방배역3번출구,,,방배역3번출구
33389,23140,임피리얼팰리스호텔.팬트힐루논현,,,임피리얼팰리스호텔.팬트힐루논현
33893,23225,대치SKVIEW아파트.서울교회,,,대치SKVIEW아파트.서울교회
34366,23308,역삼e-편한세상이아파트,,,역삼e-편한세상이아파트
34790,23385,수서119안전센터앞,,,수서119안전센터앞


In [340]:
bus2402_df2["역명3"] = bus2402_df2["역명3"].str.replace(".", "")

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\29564721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["역명3"] = bus2402_df2["역명3"].str.replace(".", "")


In [342]:
busstop_df["정류소명"] = busstop_df["정류소명"].str.replace(".", "")

In [345]:
busstop_df.head()

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입
0,100000001,1001,종로2가사거리,126.987752,37.569806,중앙차로
1,100000002,1002,창경궁서울대학교병원,126.996521,37.579433,중앙차로
2,100000003,1003,명륜3가성대입구,126.998251,37.582580,중앙차로
3,100000004,1004,종로2가삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리여운형활동터,127.001744,37.586243,중앙차로


In [350]:
for i in "방배역.서초프라자약국".split("."):
    print(busstop_df[busstop_df["정류소명"].str.contains(i)])

KeyError: 0

In [359]:
busstop_df[busstop_df["정류소명"].str.contains("방배역")].iloc[0, 3]

126.9964723757

In [360]:
busstop_df[busstop_df["정류소명"].str.contains("방배역")].iloc[0, 4]

37.4835892379

In [473]:
bus2402_df3 = bus2402_df.loc[:,("버스정류장ARS번호", "역명2")]

In [474]:
bus2402_df3 = bus2402_df3.drop_duplicates()

In [475]:
bus2402_df3.head()

,버스정류장ARS번호,역명2
0,1001,종로2가사거리
4,1002,창경궁.서울대학교병원
23,1003,명륜3가.성대입구
42,1004,종로2가.삼일교
49,1005,혜화동로터리.여운형활동터


In [476]:
def ars_coord(ars):
    try:
        a = [busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 4].iloc[0], busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 3].iloc[0]]
        return a
        
    except:
        return []

In [477]:
ars_coord = bus2402_df3["버스정류장ARS번호"].map(ars_coord)

In [478]:
bus2402_df3["ars_coord"] = ars_coord

In [479]:
bus2402_df3.head()

,버스정류장ARS번호,역명2,ars_coord
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]"
4,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]"
23,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]"
42,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]"
49,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]"


In [480]:
busstop_df

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입
0,100000001,1001,종로2가사거리,126.987752,37.569806,중앙차로
1,100000002,1002,창경궁서울대학교병원,126.996521,37.579433,중앙차로
2,100000003,1003,명륜3가성대입구,126.998251,37.582580,중앙차로
3,100000004,1004,종로2가삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리여운형활동터,127.001744,37.586243,중앙차로
...,...,...,...,...,...,...
11256,124000334,25995,우성아파트,127.139339,37.550386,일반차로
11257,124000333,25996,우성아파트,127.140046,37.550643,일반차로
11258,124000332,25997,조일약국,127.123596,37.533630,일반차로
11259,124000331,25998,성내시장,127.125497,37.536155,일반차로


In [482]:
bus2402_df3["역_list"]

0               [종로2가사거리]
4          [창경궁, 서울대학교병원]
23           [명륜3가, 성대입구]
42            [종로2가, 삼일교]
49       [혜화동로터리, 여운형활동터]
               ...       
42341           [은평공영차고지]
42352              [김포교통]
42358              [김포교통]
42364            [구미동차고지]
42367             [가좌동종점]
Name: 역_list, Length: 12701, dtype: object

In [481]:
bus2402_df3["역_list"] = bus2402_df3["역명2"].map(remove_string).str.split(".")

In [495]:
bus2402_df3["역_list"].iloc[100:120]

627               [동묘앞역5번출구]
628               [창신3동주민센터]
630      [숭인1동주민센터, 창신역4번출구]
633           [동묘역, 롯데캐슬천지인]
636                 [신설동로터리]
646                [구기동현대빌라]
649                   [광장시장]
652                  [혜화경찰서]
654    [서울역사박물관, 김구집무실(경교장)]
655              [청계3가, 관수교]
656             [청계6가, 버들다리]
657            [청계7가, 맑은내다리]
658              [청계8가, 영도교]
659                 [원남동사거리]
665                    [숭인동]
669                    [조계사]
670                 [세종문화회관]
672               [이화동(이화장)]
678                  [상명대정문]
683                  [이북오도청]
Name: 역_list, dtype: object

In [483]:
bus2402_df3.head()

,버스정류장ARS번호,역명2,ars_coord,역_list
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]",[종로2가사거리]
4,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]","[창경궁, 서울대학교병원]"
23,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]","[명륜3가, 성대입구]"
42,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]","[종로2가, 삼일교]"
49,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]","[혜화동로터리, 여운형활동터]"


In [408]:
for i in ["창경궁", "서울대학교병원"]:
    busstop_df.loc[busstop_df["정류소명"].str.contains(i, regex=False)]

In [413]:
busstop_df.loc[busstop_df["정류소명"].str.contains("창경궁", regex=False)].iloc[0, 4]

37.579432932

In [414]:
busstop_df.loc[busstop_df["정류소명"].str.contains("창경궁", regex=False)].iloc[0, 3]

126.9965206101

In [488]:
def stop_coord(stop):
    return [busstop_df.loc[busstop_df["정류소명"].str.contains(stop, regex=False)].iloc[0, 4], 
            busstop_df.loc[busstop_df["정류소명"].str.contains(stop, regex=False)].iloc[0, 3]]

In [496]:
def stoplist_coord(stoplist):
    a = []
    try:
        for i in stoplist:
            try:
                a = stop_coord(i)

            except:
                pass
    except:
        a = [""]
        
    return a

In [497]:
stop_coord = bus2402_df3["역_list"].map(stoplist_coord)

In [498]:
bus2402_df3["stop_coord"] = stop_coord

In [504]:
bus2402_df3.tail()

,버스정류장ARS번호,역명2,ars_coord,역_list,stop_coord
42341,~,은평공영차고지(가상),[],[은평공영차고지],"[37.5913387098, 126.8832624811]"
42352,~,김포교통(기점가상),[],[김포교통],[]
42358,~,김포교통(종점가상),[],[김포교통],[]
42364,~,구미동차고지(기점가상),[],[구미동차고지],[]
42367,~,가좌동종점(종점가상),[],[가좌동종점],[]


In [520]:
bus2402_df3.loc[(bus2402_df3["ars_coord"].map(lambda x: x == [])) & (bus2402_df3["stop_coord"].map(lambda x: x == []))].iloc[:60]

,버스정류장ARS번호,역명2,ars_coord,역_list,stop_coord
19553,14148,난지한강공원(야구장),[],[난지한강공원(야구장)],[]
32743,0,염곡지하차도북측(가상),[],[염곡지하차도북측],[]
33127,22981,방배역3번출구,[],[방배역3번출구],[]
34366,23308,역삼e-편한세상이아파트,[],[역삼e-편한세상이아파트],[]
38009,25302,중앙보훈병원1번출구,[],[중앙보훈병원1번출구],[]
38013,25303,중앙보훈병원1번출구,[],[중앙보훈병원1번출구],[]
38458,48087,고등동우체국.성남농협대왕지점,[],"[고등동우체국, 성남농협대왕지점]",[]
38460,48607,시흥동농협창고,[],[시흥동농협창고],[]
38461,48608,여수대교,[],[여수대교],[]
38462,48016,성남우체국,[],[성남우체국],[]
